In [ ]:
import pandas as pd

mirai = pd.read_csv("../data/blog_eda/mirai.csv")
mirai.shape

In [ ]:
import datetime

timestamps = pd.to_datetime(mirai["Time"], format="%Y-%m-%d %H:%M:%S.%f")
mirai["Time"] = timestamps
print("Timestamps")
print(timestamps)

In [ ]:
import numpy as np

interarrival_times = np.diff(timestamps) / 1000
interarrival_times

In [ ]:
interarrival_times = np.append(interarrival_times, [0])
mirai["Interarrivals"] = interarrival_times.astype(int)
mirai

In [ ]:
from matplotlib_inline.backend_inline import set_matplotlib_formats

set_matplotlib_formats("svg")

In [ ]:
%config InlineBackend.figure_format = 'svg'

In [ ]:
mirai.hist(column="Length", bins=range(50, 100))

In [ ]:
mirai.hist(column="Interarrivals")

In [ ]:
import seaborn as sns

sns.histplot(data=mirai, x="Length", binrange=(50, 100))

In [ ]:
mirai_subset = mirai.head(10000)
mirai_subset.plot.scatter(x="Length", y="Interarrivals")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(
    "Length",
    "Interarrivals",
    "",
    data=mirai_subset,
    linestyle="",
    marker="o",
    markersize=0.7,
)

In [ ]:
sns.kdeplot(data=mirai_subset, x="Length", y="Interarrivals", cmap="Reds", shade=True)
plt.title("Overplotting? Try 2D density graph", loc="left")
plt.show()

In [ ]:
import seaborn as sns

mirai_subset = mirai.sample(n=100, random_state=1)
sns.boxplot(x=mirai_subset["Length"], y=mirai_subset["Interarrivals"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

mirai_source_counts = mirai_subset["Source"].value_counts()
height = mirai_source_counts.values.tolist()
bars = tuple(mirai_source_counts.index.values)
y_pos = np.arange(len(bars))

# Create bars
plt.barh(y_pos, height)

# Create names on the x-axis
plt.yticks(y_pos, bars)

# Show graphic
plt.show()

In [ ]:
mirai_subset["Info"].str.cat(sep=" ")

In [ ]:
# Libraries
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Create a list of word
text = mirai_subset["Info"].str.cat(sep=" ")

# Create the wordcloud object
wordcloud = WordCloud(width=800, height=800, background_color="white").generate(text)

# Display the generated image:
plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)

# Show the plot
plt.show()

In [ ]:
mirai_src_dst = mirai_subset.loc[:, ["Source", "Destination"]]

In [ ]:
mirai_subset.groupby("Source", group_keys=True).apply(lambda x: x)

In [ ]:
# libraries
import seaborn as sns
import pandas as pd
import numpy as np

# Create a dataset
# mirai_source_counts = mirai_subset["Source"].value_counts()
mirai_dest_counts = mirai_subset["Destination"].value_counts()
mirai_protocol_counts = mirai_subset["Protocol"].value_counts()
mirai_length_counts = mirai_subset["Length"].value_counts()
mirai_src_port_counts = mirai_subset["Source Port"].value_counts()
mirai_dst_port_counts = mirai_subset["Dest Port"].value_counts()
mirai_interarrival_counts = mirai_subset["Interarrivals"].value_counts()
mirai_dest_counts.shape
# concatenated_array = np.stack((mirai_source_counts, mirai_dest_counts, mirai_src_port_counts, mirai_dst_port_counts, mirai_length_counts, mirai_interarrival_counts, mirai_protocol_counts), axis=1)

# bars = mirai_source_counts.index.values.tolist()
# concatenated_array
# df = pd.DataFrame(mirai_source_counts.values, columns=bars)

# # # # plot a heatmap with annotation
# sns.heatmap(df, annot=True, annot_kws={"size": 7})

In [ ]:
import hiplot as hip

# data = [{'lr': 0.001, 'loss': 10.0, 'r2': 0.8, 'optimizer': 'SGD'},
#         {'lr': 0.01, 'loss': 2.5, 'r2': 0.9, 'optimizer': 'Adam'},
#         {'lr': 0.1, 'loss': 4, 'r2': 0.86, 'optimizer': 'Adam'}]
hip.Experiment.from_iterable(mirai_subset).display()

In [ ]:
# libraries
import matplotlib.pyplot as plt
import squarify  # pip install squarify (algorithm for treemap)
import pandas as pd

# prep data
# first filter out broadcast destination
mirai_dest = mirai_subset[mirai_subset["Destination"] != "Broadcast"]
# create count and label values
mirai_dest_counts = mirai_dest["Destination"].value_counts()
sizes = mirai_dest_counts.values.tolist()
labels = list(mirai_dest_counts.index.values)

# plot it
squarify.plot(sizes=sizes, label=labels, alpha=0.8)
plt.axis("off")
plt.show()